<p align="center"><img src="https://github.com/sotopia-lab/sotopia/blob/main/figs/title.png?raw=true" alt="drawing" width="100%"/></p>

# Sotopia Tutorial Series

<a href="https://colab.research.google.com/github/sotopia-lab/sotopia/blob/main/notebooks/tutorials/1.1-setup.ipynb" target="_parent"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"> </img> </a>

The following is the first tutorial of this series to teach how to quickly build your socially intelligent agents on the sotopia platform.

## Quick links

1. Basics
    
    1.1 [Playing with Sotopia in a Google Colab Notebook](https://colab.research.google.com/drive/14hJOfzpA37PRUzdlFgiqVzUGIhhngqnz?usp=sharing)
    
    1.2 [Browsing sotopia data](https://colab.research.google.com/drive/1Gi2U3cA0KG1nekw1A0N0DZpx6gKrj8Nc?usp=sharing)

2. Build your agents (coming soon!)
3. Evaluate your agents (coming soon!)
4. Extending Sotopia (coming soon!)

Please contact [Hao Zhu](https://zhuhao.me) for any questions!

# Guide on running Redis Stack for [Sotopia](https://sotopia.world)

Author: [Hao Zhu](https://zhuhao.me)

After the release of Sotopia package, I received feedback on the reliance of Sotopia on Redis, a KV database for maintaining profiles, interaction logs, error logs, and communication between the FastAPI server and agents. When hearing about database, people often think of a huge overhead. However, I want to show you in this tutorial that setting up Redis backend for Sotopia is actually extremely easy -- just run the following blocks.

## Install Redis



In [ ]:
# @title The following block downloads Redis stack server 7.2.0, and libssl 1.1.
%%capture
!curl -fsSL https://packages.redis.io/redis-stack/redis-stack-server-7.2.0-v10.focal.x86_64.tar.gz -o redis-stack-server.tar.gz
!tar -xvf redis-stack-server.tar.gz
# Installs libssl1.1 for Ubuntu 22 source: https://stackoverflow.com/questions/72133316/libssl-so-1-1-cannot-open-shared-object-file-no-such-file-or-directory
!wget http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2.22_amd64.deb
!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2.22_amd64.deb
%pip install redis

In [ ]:
# @title Just one line of code to spin up your redis server:


!./redis-stack-server-7.2.0-v10/bin/redis-stack-server --daemonize yes

Starting redis-stack-server, database path ./redis-stack-server-7.2.0-v10/var/db/redis-stack


If the above cell results in no error message, you have successfully set up your Redis server.

In [ ]:
# @title Try it out:

import redis

client = redis.Redis(host="localhost", port=6379)
client.ping()
client.set("foo", "Yes it works.")
client.get("foo")

b'Yes it works.'

## Run Sotopia with it

To simulate an episode in Sotopia, there's one thing you will have to do: adding your `OPENAI_API_KEY` to the secret to left. Click on the little 🔑 shape button, and then toggle "Notebook access", input "OPENAI_API_KEY" under Name and the key string under Value.



In [ ]:
# @title Run the following cell to add it to the environment variable.
%%capture
%pip install sotopia
import os
from google.colab import userdata

os.environ["REDIS_OM_URL"] = "redis://:@localhost:6379"
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# @title Helper functions for adding characters and social tasks
from sotopia.database.persistent_profile import AgentProfile, EnvironmentProfile
from typing import Any


def add_agent_to_database(**kwargs: dict[str, Any]) -> None:
    agent = AgentProfile(**kwargs)
    agent.save()


def add_env_profile(**kwargs: dict[str, Any]) -> None:
    env_profile = EnvironmentProfile(**kwargs)
    env_profile.save()

In [ ]:
# @title Let's first add a character to the database. Only name is required when creating a new character


first_name = "Alice"  # @param {type:"string"}
last_name = "Smith"  # @param {type:"string"}
age = 22  # @param {type:"integer"}
occupation = "Data Scientist"  # @param {type:"string"}
gender = "Woman"  # @param ["Man", "Woman", "Nonbinary"]
gender_pronoun = "she/her"  # @param {type:"string"}
big_five = "extraversion, openness"  # @param {type:"string"}
moral_values = ["power", "achievement"]  # @param {type:"string"}
decision_making_style = "analytical"  # @param {type:"string"}
secret = "She cheated in her code interview"  # @param {type:"string"}

add_agent_to_database(
    first_name=first_name,
    last_name=last_name,
    age=age,
    occupation=occupation,
    gender=gender,
    gender_pronoun=gender_pronoun,
    big_five=big_five,
    moral_values=moral_values,
    decision_making_style=decision_making_style,
    secret=secret,
)

In [ ]:
# @title Let's add another character


first_name = "Bob"  # @param {type:"string"}
last_name = "Johnson"  # @param {type:"string"}
age = 30  # @param {type:"integer"}
occupation = "Chef"  # @param {type:"string"}
gender = "Man"  # @param ["Man", "Woman", "Nonbinary"]
gender_pronoun = "he/his"  # @param {type:"string"}
big_five = "openness, conscientiousness"  # @param {type:"string"}
moral_values = ["Benevolence"]  # @param {type:"string"}
decision_making_style = "conceptual"  # @param {type:"string"}
secret = "He has a negative net worth."  # @param {type:"string"}

add_agent_to_database(
    first_name=first_name,
    last_name=last_name,
    age=age,
    occupation=occupation,
    gender=gender,
    gender_pronoun=gender_pronoun,
    big_five=big_five,
    moral_values=moral_values,
    decision_making_style=decision_making_style,
    secret=secret,
)

In [ ]:
# @title Let's add a social tasks

scenario = 'A friend is raising a fund for the "Help the children" charity'  # @param {type:"string"}
social_goal_1 = "Ask for donation of $100"  # @param {type:"string"}
social_goal_2 = "Donate less than $10"  # @param {type:"string"}

add_env_profile(scenario=scenario, agent_goals=[social_goal_1, social_goal_2])

In [ ]:
# @title Finally, run a server:

from sotopia.samplers import UniformSampler
from sotopia.server import run_async_server


await run_async_server(
    model_dict={
        "env": "gpt-4",
        "agent1": "gpt-3.5-turbo",
        "agent2": "gpt-3.5-turbo",
    },
    sampler=UniformSampler(),
)

/usr/local/lib/python3.10/dist-packages/beartype/_util/hint/pep/utilpeptest.py:339: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Sequence[sotopia.agents.base_agent.BaseAgent[sotopia.messages.message_classes.Observation, sotopia.messages.message_classes.AgentAction]] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(
Running one batch:   0%|          | 0/1 [00:00<?, ?it/s]

Is the agent omniscient? False

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



AgentProfile(
    pk='01J05GYPQX5AZ1QNEFS3V3BTEF',
    first_name='Bob',
    last_name='Johnson',
    age=30,
    occupation='Chef',
    gender='Man',
    gender_pronoun='he/his',
    public_info='',
    big_five='openness, conscientiousness',
    moral_values=['Benevolence'],
    schwartz_personal_values=[],
    personality_and_values='',
    decision_making_style='conceptual',
    secret='He has a negative net worth.',
    model_id='',
    mbti=''
)

AgentProfile(
    pk='01J05GYPQNBPB86CEMQTXZ885J',
    first_name='Alice',
    last_name='Smith',
    age=22,
    occupation='Data Scientist',
    gender='Woman',
    gender_pronoun='she/her',
    public_info='',
    big_five='extraversion, openness',
    moral_values=['power', 'achievement'],
    schwartz_personal_values=[],
    personality_and_values='',
    decision_making_style='analytical',
    secret='She cheated in her code interview',
    model_id='',
    mbti=''
)

Here is the context of this interaction:
Scenario: A friend is raising a fund for the "Help the children" charity
Participants: Bob Johnson and Alice Smith
Bob Johnson's background: Bob Johnson is a 30-year-old male chef. he/his pronouns.  Personality and values 
description:  Bob's secrets: He has a negative net worth.
Alice Smith's background: Unknown
Bob Johnson's goal: Ask for donation of $100
Alice Smith's goal: Unknown
Conversation Starts:


Here is the context of this interaction:
Scenario: A friend is raising a fund for the "Help the children" charity
Participants: Bob Johnson and Alice Smith
Bob Johnson's background: Unknown
Alice Smith's background: Alice Smith is a 22-year-old female data scientist. she/her pronouns.  Personality and 
values description:  Alice's secrets: She cheated in her code interview
Bob Johnson's goal: Unknown
Alice Smith's goal: Donate less than $10
Conversation Starts:

Bob Johnson said: "Hey Alice, how are you doing today?"

...

Bob Johnson said: "That's completely understandable, Alice. I appreciate your honesty. There are actually other 
ways you can support the cause. You can help spread the word about the fundraising campaign on your social media 
platforms or even volunteer your time at one of their events. Any support, big or small, is valuable and greatly 
appreciated."

The reasoning is:
Environment comments: terminated: The conversation stales for too long;

The rewards are:
Agent 1: 0.0
Agent 2: 0.0

Running one batch: 100%|██████████| 1/1 [00:14<00:00, 14.88s/it]


[[('Environment',
   'Bob Johnson',
   Observation(last_turn='Here is the context of this interaction:\nScenario: A friend is raising a fund for the "Help the children" charity\nParticipants: Bob Johnson and Alice Smith\nBob Johnson\'s background: Bob Johnson is a 30-year-old male chef. he/his pronouns.  Personality and values description:  Bob\'s secrets: He has a negative net worth.\nAlice Smith\'s background: Unknown\nBob Johnson\'s goal: Ask for donation of $100\nAlice Smith\'s goal: Unknown', turn_number=0, available_actions=['leave', 'none', 'action', 'speak', 'non-verbal communication'])),
  ('Environment',
   'Alice Smith',
   Observation(last_turn='Here is the context of this interaction:\nScenario: A friend is raising a fund for the "Help the children" charity\nParticipants: Bob Johnson and Alice Smith\nBob Johnson\'s background: Unknown\nAlice Smith\'s background: Alice Smith is a 22-year-old female data scientist. she/her pronouns.  Personality and values description:  Alic

### A note on what is running under the hood

After you added the characters and scenarios to the database, the `run_async_server` samples a pair of characters and one scenario to run the simulation, and then evaluate the simulation. All of the sampling, interaction simulation, and evaluation are handled by Sotopia. You could easily tweak them through editing `run_async_server`, or changing the LLMs, or the sampler according to your own need.

## Conclusion

In this very short tutorial, I demonstrated the whole pipeline of setting up Redis server, installing sotopia, adding characters and social tasks, and simulating + evaluating the social interaction between two GPT-3.5 agents with GPT-4. This tutorial intends to serve as the first quick example for you. The only thing you have to do besides running all of the cells is add your own OpenAI key<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1). You could also use other LLMs supported by LiteLLM even your own API endpoint. Let me know if you want me to explain more.

<a name="cite_note-1"></a> [1](#cite_ref-1): *sorry I cannot afford to share mine.*